# Object Detection - (기본-6) Ensemble
#### Ensemble
앙상블은 최종 아웃풋의 품질과 가장 직접적으로 연관이 있고, 시간 대비 좋은 결과를 낼 수 있는 방법입니다! <br>
지금까지 학습시킨 모델들을 Sample Submission을 이용해 앙상블 코드를 작성하고, 다양한 결과를 직접 확인해봅시다
<br> 🌟Ensemble의 자세한 내용은 09강: Ready for Competition 강의를 참고합니다.

## 대회 데이터셋 구성
Custom 데이터를 구현하여 대회 데이터셋에 Ensemble 방법을 적용해봅니다. <br>
데이터셋의 자세한 개요는 [대회 플랫폼](https://stages.ai/competitions/325/data/overview)의 데이터 설명을 참고합니다.
> Copyright: CC BY 2.0

### dataset
    ├── train.json
    ├── test.json
    ├── train
    └── test

In [1]:
!pip install ensemble_boxes

  Obtaining dependency information for ensemble_boxes from https://files.pythonhosted.org/packages/0e/5b/58e47cd45fc18da37205a80689606e0e203810f1beadbdaae620f491892b/ensemble_boxes-1.0.9-py3-none-any.whl.metadata
  Obtaining dependency information for numba from https://files.pythonhosted.org/packages/79/58/cb4ac5b8f7ec64200460aef1fed88258fb872ceef504ab1f989d2ff0f684/numba-0.60.0-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata
  Obtaining dependency information for llvmlite<0.44,>=0.43.0dev0 from https://files.pythonhosted.org/packages/c6/21/2ffbab5714e72f2483207b4a1de79b2eecd9debbf666ff4e7067bcc5c134/llvmlite-0.43.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 3.8 MB/s eta 0:00:0000:0100:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 MB 12.7 MB/s eta 0:00:0000:0100:01


In [2]:
import pandas as pd
import numpy as np
from ensemble_boxes import nms
from pycocotools.coco import COCO

In [5]:
# ensemble csv files
submission_files = ['../sample_submission/faster_rcnn_torchvision_submission.csv',
                   '../sample_submission/faster_rcnn_mmdetection_submission.csv']
submission_df = [pd.read_csv(file) for file in submission_files]

In [6]:
image_ids = submission_df[0]['image_id'].tolist()

In [7]:
# ensemble 할 file의 image 정보를 불러오기 위한 json
annotation = '../../dataset/test.json'
coco = COCO(annotation)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


###💡 Ensemble 구현하기
한글 주석을 참고하여 `'코드 추가'` 에 코드를 추가해 주시면 됩니다! (어려우신 경우 강의와 정답코드를 참고해서 추가해주세요~)
1. submission_files에서 boxes, scores, labels 정보 가져오기
2. NMS를 위해 boxes 정보를 image 크기(height, width)로 정규화
3. NMS 함수를 사용하여 ensemble 수행 후 결과 파일 생성


In [8]:
prediction_strings = []
file_names = []
# ensemble 시 설정할 iou threshold 이 부분을 바꿔가며 대회 metric에 알맞게 적용해봐요!
iou_thr = 0.4

# 각 image id 별로 submission file에서 box좌표 추출
for i, image_id in enumerate(image_ids):
    prediction_string = ''
    boxes_list = []
    scores_list = []
    labels_list = []
    image_info = coco.loadImgs(i)[0]
    # 각 submission file 별로 prediction box좌표 불러오기
    for df in submission_df:
        predict_string = df[df['image_id'] == image_id]['PredictionString'].tolist()[0]
        predict_list = str(predict_string).split()

        if len(predict_list)==0 or len(predict_list)==1:
            continue

        predict_list = np.reshape(predict_list, (-1, 6))
        box_list = []

        for box in predict_list[:, 2:6].tolist():
            # box의 각 좌표를 float형으로 변환한 후 image의 넓이와 높이로 각각 정규화
            image_width = image_info['width']
            image_height = image_info['height']
            box[0] = float(box[0]) / image_width
            box[1] = float(box[1]) / image_height
            box[2] = float(box[2]) / image_width
            box[3] = float(box[3]) / image_height
            box_list.append(box)

        boxes_list.append(box_list)
        scores_list.append(list(map(float, predict_list[:, 1].tolist())))
        labels_list.append(list(map(int, predict_list[:, 0].tolist())))

    # 예측 box가 있다면 이를 ensemble 수행
    if len(boxes_list):
        # ensemble_boxes에서 import한 nms()를 사용하여 NMS 계산 수행
        # 👉 위의 코드에서 만든 정보들을 함수에 간단하게 적용해보세요!
        # nms에 필요한 인자: [NMS할 box의 lists, confidence score의 lists, label의 list, iou에 사용할 threshold]
        boxes, scores, labels = nms(boxes_list, scores_list, labels_list, iou_thr=iou_thr)
        for box, score, label in zip(boxes, scores, labels):
            prediction_string += str(label) + ' ' + str(score) + ' ' + str(box[0] * image_info['width']) + ' ' + str(box[1] * image_info['height']) + ' ' + str(box[2] * image_info['width']) + ' ' + str(box[3] * image_info['height']) + ' '

    prediction_strings.append(prediction_string)
    file_names.append(image_id)

In [10]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv('../sample_submission/submission_ensemble2.csv', index=False)
submission.head()

,PredictionString,image_id
0,0 0.24645094573497772 242.52542114257812 683.2...,test/0000.jpg
1,0 0.45437225699424744 713.5231323242188 648.64...,test/0001.jpg
2,0 0.3595143258571625 655.670166015625 288.0221...,test/0002.jpg
3,0 0.45486128330230713 0.0 442.3086242675781 17...,test/0003.jpg
4,0 0.6036637425422668 193.4078826904297 346.774...,test/0004.jpg


### Required Package
테스트 실행 환경 (google colab 기준)
```
numpy==1.26.4
torch==2.3.1+cu121
pandas==2.1.4
pycocotools==2.0.8
```



### Reference
https://github.com/ZFTurbo/Weighted-Boxes-Fusion

###**콘텐츠 라이선스**

<font color='red'><b>**WARNING**</b></font> : **본 교육 콘텐츠의 지식재산권은 재단법인 네이버커넥트에 귀속됩니다. 본 콘텐츠를 어떠한 경로로든 외부로 유출 및 수정하는 행위를 엄격히 금합니다.** 다만, 비영리적 교육 및 연구활동에 한정되어 사용할 수 있으나 재단의 허락을 받아야 합니다. 이를 위반하는 경우, 관련 법률에 따라 책임을 질 수 있습니다.
